# ASCAT data loading Hans Lievens

Loading some useful libraries

In [1]:
import numpy as np

### Received information about dataset (email-conversation)

Each cells contains two files: an
index (.idx) and a data (.dat) file.

The index file contains the grid point numbers and the data file the
data records. First read the .idx file, choose the grid point you want
to read, use the exact same rows of this grid point for the .dat file
to read the data.

When you read the files you have to skip the first 208 bytes, this is
the file header we are using internally to select the correct
template.

For reading the .idx file the date type template looks like
struct = np.dtype([('gpi', np.int32)])

and for the .dat file the data type template looks like
struct = np.dtype([('jd', np.double),  ('sig', np.float32),
('sig_noise', np.float32),    ('dir', np.dtype('S1')),    ('pdb',
np.ubyte),    ('azcorr_flag', np.dtype([('f', np.ubyte),  ('m',
np.ubyte),  ('a', np.ubyte)]))],

### First draft reading of the binary data

In the following section, I'll test the reading for the following dataset/file:

In [2]:
filename = "1323"

#### INDEX FILE

We have to skip the  first 208 bytes to avoid the reading of the header. 

In [3]:
struct1 = np.dtype([('gpi', np.int32)])

In [4]:
f = open("".join(["./ASCAT/", filename, ".idx"]), "rb")  # reopen the file
f.seek(208, os.SEEK_SET)  # seek

idx_data = np.fromfile(f, dtype=struct1).astype('int32') # astype erbij om de omzetting te verzekeren

In [5]:
idx_data.shape # grootte evalueren

(4488295,)

In [6]:
idx_data[:50]

array([2301795, 2301795, 2301795, 2301795, 2301795, 2301795, 2301795,
       2301795, 2301795, 2301795, 2301795, 2301795, 2301795, 2301795,
       2301795, 2301795, 2301795, 2301795, 2301795, 2301795, 2301795,
       2301795, 2301795, 2301795, 2301795, 2301795, 2301795, 2301795,
       2301795, 2301795, 2301795, 2301795, 2301795, 2301795, 2301795,
       2301795, 2301795, 2301795, 2301795, 2301795, 2301795, 2301795,
       2301795, 2301795, 2301795, 2301795, 2301795, 2301795, 2301795,
       2301795], dtype=int32)

Ik lees hier een lijst van getallen in, telkens een cluster van -wat ik veronderstel- grid points; laat ond even de unieke waarden nagaan:

In [7]:
np.unique(idx_data)

array([2301795, 2301797, 2301801, ..., 2493005, 2493009, 2493013], dtype=int32)

In [8]:
np.unique(idx_data).size

1379

Met andere woorden 1379 grid points zijn hier in meegegeven...

#### DAT-FILE data zelf

Inlezen, met de gegeven byte header en de structuur die was meegeleverd:

In [9]:
struct2 = np.dtype([('jd', np.double),  ('sig', np.float32), ('sig_noise', np.float32), 
                   ('dir', np.dtype('S1')),    ('pdb',np.ubyte), ('azcorr_flag', 
                                                                  np.dtype([('f', np.ubyte),  
                                                                            ('m',np.ubyte),  
                                                                            ('a', np.ubyte)]))])

In [10]:
f = open(''.join(["./ASCAT/", filename, ".dat"]), "rb")  # reopen the file
f.seek(208, os.SEEK_SET)  # seek

data = np.fromfile(f, dtype=struct2)

In [11]:
data

array([ (2454102.8887586812, -9.16275691986084, 0.08637592941522598, 'D', 54, (2, 2, 2)),
       (2454103.9439236117, -9.11705493927002, 0.08362860232591629, 'D', 54, (2, 2, 2)),
       (2454104.3495876626, -9.267045021057129, 0.08337954431772232, 'A', 54, (2, 2, 2)),
       ...,
       (2457022.369726563, -8.836539268493652, 0.07853401452302933, 'A', 54, (2, 2, 2)),
       (2457022.8782986, -8.78021240234375, 0.08376690745353699, 'D', 54, (2, 2, 2)),
       (2457022.947786447, -8.758130073547363, 0.08410192281007767, 'D', 54, (2, 2, 2))], 
      dtype=[('jd', '<f8'), ('sig', '<f4'), ('sig_noise', '<f4'), ('dir', 'S1'), ('pdb', 'u1'), ('azcorr_flag', [('f', 'u1'), ('m', 'u1'), ('a', 'u1')])])

Voorbeeld van 1 element:

In [12]:
data[1000]

(2455067.402756065, -9.999258995056152, 0.08587104082107544, 'A', 54, (2, 2, 2))

Ik had al vernomen dat het tweede getal een backscatterwaarde zou zijn tussen 0 en -30, dat lijkt me te kloppen. Bovendien lijkt de eerste waarde een datum te kunnen zijn...


### Extract data from a gridpoint

Mijn hypothese op basis van zijn uitleg en bovenstaande data: 
* Kies een nummer uit de unieke waarden
* selecteer alle rijen met die waarde en onthou de indexen
* selecteer de rijen van die indexen uit de .dat-file

Zou dat steek houden? - ik probeer het alvast uit (We hebben nu *idx_data* en de *data* arrays om mee te werken):

In [13]:
# We kiezen 1 specifiek gridpoint
grid_point = 2301829

In [14]:
# selecteer de 'rijen/posities' voor dat grid point
indxs_point = np.where(idx_data == grid_point)

In [15]:
# haal die rijen uit de data-file
my_selection = data[indxs_point]

In [16]:
my_selection

array([ (2454102.8887369796, -9.05182933807373, 0.0817079022526741, 'D', 54, (2, 2, 2)),
       (2454103.943836806, -9.10305404663086, 0.08119107037782669, 'D', 54, (2, 2, 2)),
       (2454104.34952257, -9.057757377624512, 0.07795611023902893, 'A', 54, (2, 2, 2)),
       ...,
       (2457021.893793403, -9.648776054382324, 0.07921148091554642, 'D', 54, (2, 2, 2)),
       (2457022.9488281254, -9.93057918548584, 0.08341851830482483, 'D', 54, (2, 2, 2)),
       (2457023.3545138896, -10.14028549194336, 0.07756844907999039, 'A', 54, (2, 2, 2))], 
      dtype=[('jd', '<f8'), ('sig', '<f4'), ('sig_noise', '<f4'), ('dir', 'S1'), ('pdb', 'u1'), ('azcorr_flag', [('f', 'u1'), ('m', 'u1'), ('a', 'u1')])])

Conversie naar matlab-file om er daar mee te werken:

In [17]:
import scipy.io as sio

In [18]:
my_selection

array([ (2454102.8887369796, -9.05182933807373, 0.0817079022526741, 'D', 54, (2, 2, 2)),
       (2454103.943836806, -9.10305404663086, 0.08119107037782669, 'D', 54, (2, 2, 2)),
       (2454104.34952257, -9.057757377624512, 0.07795611023902893, 'A', 54, (2, 2, 2)),
       ...,
       (2457021.893793403, -9.648776054382324, 0.07921148091554642, 'D', 54, (2, 2, 2)),
       (2457022.9488281254, -9.93057918548584, 0.08341851830482483, 'D', 54, (2, 2, 2)),
       (2457023.3545138896, -10.14028549194336, 0.07756844907999039, 'A', 54, (2, 2, 2))], 
      dtype=[('jd', '<f8'), ('sig', '<f4'), ('sig_noise', '<f4'), ('dir', 'S1'), ('pdb', 'u1'), ('azcorr_flag', [('f', 'u1'), ('m', 'u1'), ('a', 'u1')])])

In [19]:
sio.savemat("data_hans", {''.join(['grid_point1_',str(grid_point)]):my_selection, ''.join(['grid_point2_',str(grid_point)]):my_selection})

Hiermee verkrijgen we een .mat-file die je kan 'load'-en in Matlab en als structuur beschikbaar is...

## Bring this together in a single functionality

Learnt all the things above, I made a function to do the analysis above in a specific function written in de file *convert_ascat.py*:

In [24]:
from convert_ascat import convert_ascat_to_matlab

Test the functionality as a function:

**Extracting a single grid point, by naming the ID**

In [25]:
my_data = convert_ascat_to_matlab("./ASCAT/1323", grid_point_id=2301795, byte2skip=208)

data loaded
working on grid point 2301795


**Extracting all grid points, all in separate files**

In [23]:
all_data = convert_ascat_to_matlab("./ASCAT/1323", grid_point_id='all', byte2skip=208)

**So, now?**

A second file, called *convert_ascat.py* is available to do this from the command line, using the filename as an extra argument, just make sure you have the python file executable. 

The dependencies are numpy and scipy. Easiest way of using this yourself is to install Anaconda, https://www.continuum.io/downloads, which will provide you these libraries (together with some other useful libraries) to make it possible.

Another option is to install python/numpy/scipy itself manually; or use Miniconda and create your own set of library combinations; install http://conda.pydata.org/miniconda.html and create environment by putting the following command in the command line:

        conda create -n myenvname python=2.7 numpy scipy

Om de conversie-tool vanaf de command line uit te voeren doen je:

            python convert_ascat.py ./ASCAT/1323 2301795

voor een specifiek element, of

            python convert_ascat.py ./ASCAT/1322 all

om alle grid points in de file te converteren (kan even duren ;-). Ik neem voor dit voorbeeld hierbij aan dat de python file in een folder staat waarin de ASCAT folder met de data in staat

Veel succes ermee en laat maar weten als je ergens vast loopt,

mvg,

Stijn